# Домашнее задание 4
### Выполнила: Кудрявцева Полина Дмитриевна

In [1]:
# импорт библиотек
import pandas as pd
import sqlite3
from datetime import datetime as dt

## Задача 1

**Необходимо скачать CSV-файл - «transactions_for_dz2.csv», создать таблицу transaction_bd со всеми полями, загрузить данные из файла в таблицу.**

**Также необходимо скачать второй CSV-файл - «client_info.csv», создать таблицу customer_bd со всеми полями, загрузить данные из файла в таблицу**



In [2]:
# Прочитаем данные из файлов и посмотрим на них
transactions_df = pd.read_csv('./data/transactions_for_dz2.csv')
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66


In [3]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   TRANSACTION_ID  1048575 non-null  int64  
 1   TX_DATETIME     1048575 non-null  object 
 2   CUSTOMER_ID     1048575 non-null  int64  
 3   TERMINAL_ID     1048575 non-null  int64  
 4   TX_AMOUNT       1048575 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 40.0+ MB


In [4]:
# Прочитаем данные из файлов и посмотрим на них
clients_df = pd.read_csv('./data/client_info.csv', sep=';')
clients_df.head()

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
0,2015-07-16,2021-01-01,Olivia,1979,2213
1,2015-07-17,2021-01-01,Emma,1979,1148
2,2015-07-18,2021-01-01,Charlotte,1979,2293
3,2015-07-19,2021-01-01,Amelia,1979,1867
4,2015-07-20,2021-01-01,Ava,1979,1767


In [5]:
clients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4988 entries, 0 to 4987
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   START_DT     4988 non-null   object
 1   END_DT       4988 non-null   object
 2   CLIENT_NAME  4988 non-null   object
 3   YEAR_BIRTH   4988 non-null   int64 
 4   CLIENT_ID    4988 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 195.0+ KB


In [6]:
# Подключимся (и при отсутствии создадим) к базе данных Transactions.db
con = sqlite3.connect("./data/Transactions.db")

In [7]:
# Инициируем курсор
cur = con.cursor()

In [8]:
# Напишем SQL-запрос для создания таблиц transaction_bd и customer_bd со всеми полями из CSV файлов и выполним его
create_transactuins = '''
CREATE TABLE transaction_bd(
    TRANSACTION_ID integer NOT NULL PRIMARY KEY,
    TX_DATETIME datetime NOT NULL,
    CUSTOMER_ID integer NOT NULL,
    TERMINAL_ID integer NOT NULL,
    TX_AMOUNT real NOT NULL
);
'''
create_customers = '''
CREATE TABLE customer_bd(
    CLIENT_ID integer NOT NULL PRIMARY KEY,
    CLIENT_NAME text NOT NULL,
    YEAR_BIRTH integer NOT NULL,
    START_DT datetime NOT NULL,
    END_DT datetime NOT NULL
);
'''

cur.execute(create_transactuins)
cur.execute(create_customers)
con.commit()

In [9]:
# Запишем данные из датафреймов в таблицы
transactions_df.to_sql("transaction_bd", con, if_exists="replace", index=False)
clients_df.to_sql("customer_bd", con, if_exists="replace", index=False)

4988

## Задача 2 - SQL-запросы к БД
**a. Вывести список всех клиентов, у которых между двумя ближайшими транзакциями был перерыв больше 35 дней хотя бы один раз (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).**

In [29]:
current_dt = '2023-05-01' #dt.strftime(dt.now(), '%Y-%m-%d')
# Напишем SQL-запрос
query_a = f'''
WITH cte AS (
SELECT CUSTOMER_ID, TX_DATETIME,
    LAG(TX_DATETIME, 1, 0) OVER(PARTITION BY CUSTOMER_ID ORDER BY TX_DATETIME) AS PREV_DATETIME,
    CAST(julianday(TX_DATETIME) - julianday(LAG(TX_DATETIME, 1, 0) OVER(PARTITION BY CUSTOMER_ID ORDER BY TX_DATETIME)) as int) AS DATE_DIFF
FROM transaction_bd t
JOIN customer_bd c ON t.CUSTOMER_ID=c.CLIENT_ID
WHERE c.END_DT>='{current_dt}')

SELECT DISTINCT CUSTOMER_ID
FROM cte
WHERE DATE_DIFF > 35 AND PREV_DATETIME <> 0
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_a = pd.read_sql(query_a, con)
df_a.head(10)

,CUSTOMER_ID
0,24
1,480
2,812
3,896
4,1129
5,1299
6,1334
7,1459
8,1637
9,1923


**b. Вывести список клиентов, у которых максимальная сумма транзакции (имеется ввиду максимальное значение TX_AMOUNT) больше 50000. В ответе должен быть id клиента, максимальное значение суммы транзакций, минимальное значение суммы транзакций (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).**

In [32]:
# Напишем SQL-запрос
query_b = f'''
SELECT CUSTOMER_ID, MAX(TX_AMOUNT) AS MAX_AMOUNT, MIN(TX_AMOUNT) AS MIN_AMOUNT
FROM transaction_bd t
JOIN customer_bd c ON t.CUSTOMER_ID=c.CLIENT_ID
WHERE c.END_DT>='{current_dt}'
GROUP BY CUSTOMER_ID
HAVING MAX_AMOUNT > 50000;
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_b = pd.read_sql(query_b, con)
df_b.head(10)

,CUSTOMER_ID,MAX_AMOUNT,MIN_AMOUNT
0,3013,77212.50,1.46
1,3494,53213.00,85.76
2,4253,51937.25,37.95


**c. Вывести топ-10 клиентов (сортируя их по убыванию id), у которых сумма дневных транзакций больше суммы вечерних транзакций Условимся, что дневными считаются транзакции, которые были произведены до 12 часов, а вечерними - после 12 часов включительно. (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).**

In [35]:
# Напишем SQL-запрос
query_c = f'''
SELECT CUSTOMER_ID,
SUM(CASE WHEN time(TX_DATETIME)<'12:00:00' THEN TX_AMOUNT END) AS DAY_AMOUNT,
SUM(CASE WHEN time(TX_DATETIME)>='12:00:00' THEN TX_AMOUNT END) AS EVENING_AMOUNT
FROM transaction_bd t
JOIN customer_bd c ON t.CUSTOMER_ID=c.CLIENT_ID
WHERE c.END_DT>='{current_dt}'
GROUP BY CUSTOMER_ID
HAVING DAY_AMOUNT > EVENING_AMOUNT
ORDER BY CUSTOMER_ID DESC
LIMIT 10;
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_c = pd.read_sql(query_c, con)
df_c.head(10)

,CUSTOMER_ID,DAY_AMOUNT,EVENING_AMOUNT
0,4997,68153.52,63135.10
1,4993,43648.09,41244.55
2,4990,86921.94,86536.14
3,4987,12239.39,11547.78
4,4983,109737.94,97444.22
5,4982,45214.41,40213.19
6,4978,3210.61,3001.03
7,4976,71894.97,61467.88
8,4973,65297.68,64986.58
9,4972,3092.83,2702.23


**d. Найти для каждого клиента день, когда у него была максимальная сумма транзакции, то есть вывести клиента, дату транзакции, сумму транзакции (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).**

In [42]:
# Напишем SQL-запрос
query_d = f'''
WITH t AS (
SELECT CUSTOMER_ID,
       strftime('%Y-%m-%d',t.TX_DATETIME) AS TX_DATE,
       SUM(TX_AMOUNT) AS TX_AMOUNT
FROM transaction_bd t
JOIN customer_bd c ON t.CUSTOMER_ID=c.CLIENT_ID
WHERE c.END_DT>='{current_dt}'
GROUP BY CUSTOMER_ID, TX_DATE
),
cte AS (     
SELECT CUSTOMER_ID, MAX(TX_AMOUNT) AS MAX_AMOUNT
FROM t
GROUP BY CUSTOMER_ID
)
SELECT cte.CUSTOMER_ID, t.TX_DATE, cte.MAX_AMOUNT
FROM cte 
JOIN t ON t.CUSTOMER_ID=cte.CUSTOMER_ID
     AND t.TX_AMOUNT = cte.MAX_AMOUNT
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_d = pd.read_sql(query_d, con)
df_d.head(10)

,CUSTOMER_ID,TX_DATE,MAX_AMOUNT
0,0,2023-02-01,4084.94
1,1,2023-01-28,19318.16
2,2,2023-04-14,6190.41
3,5,2023-03-17,4906.86
4,6,2023-04-12,1504.53
5,7,2023-03-04,2220.69
6,8,2023-03-14,197.76
7,9,2023-03-26,7219.31
8,10,2023-01-29,36196.50
9,11,2023-03-23,965.22


**e. Вычислить интегральное распределение суммы транзакций для каждого дня за весь период, то есть вывести дату транзакции, сумму транзакции и интегральное распределение по сумме (не учитываем что клиент может быть недействующим)**

In [44]:
# Напишем SQL-запрос
query_e = f'''
WITH cte AS (
SELECT strftime('%Y-%m-%d',t.TX_DATETIME) AS TX_DATE,
       SUM(TX_AMOUNT) AS TX_AMOUNT
FROM transaction_bd t
GROUP BY TX_DATE
)
SELECT TX_DATE, TX_AMOUNT, CUME_DIST() OVER(ORDER BY TX_AMOUNT) AS distribution
FROM cte 
ORDER BY TX_DATE
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_e = pd.read_sql(query_e, con)
df_e.head(10)

,TX_DATE,TX_AMOUNT,distribution
0,2023-01-01,4827656.26,0.018182
1,2023-01-02,4862551.41,0.027273
2,2023-01-03,5058973.71,0.209091
3,2023-01-04,4938142.47,0.036364
4,2023-01-05,5002954.23,0.109091
5,2023-01-06,5020241.79,0.136364
6,2023-01-07,4974352.96,0.063636
7,2023-01-08,5020491.33,0.145455
8,2023-01-09,5098359.96,0.309091
9,2023-01-10,5312800.13,0.927273
